## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from sklearn.naive_bayes import GaussianNB
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

## Importing the dataset

In [2]:
data = pd.read_excel('live_data_alb2.0_ml.xlsx',ignore_index=True)

In [3]:
df = data.drop_duplicates()

## Exploratory data analysis

In [4]:
df.shape

(10700, 14)

In [5]:
df.head(2)

aiscoreid                                      aiscorestring  \
0       4308  [{"test_score": ""80, A104 - Review/Analyze, 8...   
1       4317  [{"b_score": "NA"},{"p_score": ""-p50""},{"o_s...   

                                          aikeywords  activityfeesexpenseid  \
0  ADMINISTRATIVE,Approved_By#Z,Approved_By#Z,App...                   8027   
1  ADMINISTRATIVE,Approved_By#Z,Approved_By#Z,App...                   8036   

  type activitycode taskcode  \
0    F         A104     L120   
1    F         A104     L120   

                                         description    source  admin_s100  \
0  Review additional documents to produce in resp...  fbt_live           0   
1  Review emails from/to E. Flowers re edits to s...  fbt_live           0   

   para_s100  bb_s100  bb_s75  ot_s100  
0          1        0       0        0  
1          0        0       0        0

In [6]:
df['description'] = df['description'].str.lower()

In [7]:
df["admin_s100"].value_counts()

0    10268
1      432
Name: admin_s100, dtype: int64

In [8]:
df["para_s100"].value_counts()

0    9221
1    1479
Name: para_s100, dtype: int64

In [9]:
print(df["bb_s100"].value_counts())

0    10692
1        8
Name: bb_s100, dtype: int64


In [10]:
df["bb_s75"].value_counts()
df["ot_s100"].value_counts()

0    10547
1      153
Name: ot_s100, dtype: int64

In [11]:
df["ot_s100"].value_counts()

0    10547
1      153
Name: ot_s100, dtype: int64

**Paralegal Task**

In [12]:
para = df[["description","para_s100"]]
para_y = para[para["para_s100"] == 1]
para_y = para_y.drop_duplicates()
para_n = para[para["para_s100"] == 0]
para_n = para_n.drop_duplicates()

In [13]:
print(para_y.shape)
print(para_n.shape)

(728, 2)
(3702, 2)


In [14]:
para_n_ran = para_n.sample(n = len(para_y),replace = False)

In [15]:
p = pd.concat([para_y,para_n_ran],axis = 0)

In [16]:
print(p.shape)

(1456, 2)


## Feature engineering

In [17]:
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm

f = feature_extraction.text.CountVectorizer(stop_words = 'english')
X = f.fit_transform(p["description"])
np.shape(X)

(1456, 2318)

## Splitting the dataset into the Training set and Test set

In [18]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, p["para_s100"], test_size=0.30, random_state=42)
print([np.shape(X_train), np.shape(X_test)])

[(1019, 2318), (437, 2318)]


## Training different classifier

**Naive bayes classifier**

We train different bayes models changing the regularization parameter  α .

We evaluate the accuracy, recall and precision of the model with the test set.

In [19]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 

Let's see the first 10 learning models and their metrics

In [20]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

alpha  Train Accuracy  Test Accuracy  Test Recall  Test Precision
0  0.00001        0.926398       0.702517     0.772321        0.686508
1  0.11001        0.907753       0.741419     0.799107        0.724696
2  0.22001        0.900883       0.755149     0.812500        0.736842
3  0.33001        0.896958       0.757437     0.812500        0.739837
4  0.44001        0.888126       0.762014     0.821429        0.741935
5  0.55001        0.886163       0.764302     0.821429        0.744939
6  0.66001        0.883219       0.759725     0.821429        0.738956
7  0.77001        0.879293       0.764302     0.830357        0.741036
8  0.88001        0.873405       0.766590     0.839286        0.740157
9  0.99001        0.869480       0.768879     0.839286        0.743083

I select the model with the most test precision

In [21]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

alpha             0.550010
Train Accuracy    0.886163
Test Accuracy     0.764302
Test Recall       0.821429
Test Precision    0.744939
Name: 5, dtype: float64

**Confusion matrix with naive bayes classifier**

In [22]:
m_confusion_test = metrics.confusion_matrix(y_test, bayes.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

Predicted 0  Predicted 1
Actual 0          130           83
Actual 1           22          202

**Support Vector**

We are going to apply the same reasoning applying the support vector machine model with the gaussian kernel.

We train different models changing the regularization parameter C.

We evaluate the accuracy, recall and precision of the model with the test set.

In [23]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [24]:
list_C = np.arange(500, 2000, 100) #100000
score_train = np.zeros(len(list_C))
score_test = np.zeros(len(list_C))
recall_test = np.zeros(len(list_C))
precision_test= np.zeros(len(list_C))
count = 0
for C in list_C:
    svc = svm.SVC(C=C)
    svc.fit(X_train, y_train)
    score_train[count] = svc.score(X_train, y_train)
    score_test[count]= svc.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, svc.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, svc.predict(X_test))
    count = count + 1 

Let's see the first 10 learning models and their metrics

In [25]:
matrix = np.matrix(np.c_[list_C, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['C', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

C  Train Accuracy  Test Accuracy  Test Recall  Test Precision
0   500.0        0.972522       0.832952     0.861607        0.821277
1   600.0        0.973503       0.823799     0.848214        0.815451
2   700.0        0.974485       0.826087     0.848214        0.818966
3   800.0        0.974485       0.821510     0.848214        0.811966
4   900.0        0.975466       0.819222     0.843750        0.811159
5  1000.0        0.978410       0.823799     0.848214        0.815451
6  1100.0        0.979392       0.812357     0.839286        0.803419
7  1200.0        0.981354       0.821510     0.848214        0.811966
8  1300.0        0.983317       0.821510     0.848214        0.811966
9  1400.0        0.983317       0.819222     0.848214        0.808511

In [26]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

C                 500.000000
Train Accuracy      0.972522
Test Accuracy       0.832952
Test Recall         0.861607
Test Precision      0.821277
Name: 0, dtype: float64

**Confusion matrix with support vector machine classifier**

In [27]:
m_confusion_test = metrics.confusion_matrix(y_test, svc.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

Predicted 0  Predicted 1
Actual 0          163           50
Actual 1           37          187

## Predicting a new result

In [28]:
text = ["analyze supplemental document production and the purpose is to determine document necessary to produce"]

X = f.transform(text) # mapping

In [29]:
res = svc.predict(X)

In [30]:
x = res[0]
if x == 1:
    print("This is a paralegal task")
else:
    print("This is not a paralegal task")

This is a paralegal task
